In [77]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from StatusMsg import StatusMsg
import json
import os
import io
import datetime
import timedelta

In [78]:
def india(state,date):
    # URL = "https://www.mygov.in/corona-data/covid19-statewise-status/"
    # file_name, headers = urllib.request.urlretrieve(URL)
    soup = BeautifulSoup(open("../INPUT/"+date+"/TT_State.html", encoding="utf8"), "html.parser")

    # sum_URL = "https://www.mygov.in/covid-19"
    # sum_file_name, sum_headers = urllib.request.urlretrieve(sum_URL)
    sum_soup = BeautifulSoup(open("../INPUT/"+date+"/TT.html", encoding="utf8"), "html.parser")

    STATES = soup.find_all("div", {"class": "field field-name-field-select-state field-type-list-text field-label-above"})
    CONFIRMED = soup.find_all("div", {"class": "field field-name-field-total-confirmed-indians field-type-number-integer field-label-above"})
    CURED_DISCHARGED = soup.find_all("div", {"class": "field field-name-field-cured field-type-number-integer field-label-above"})
    DEATH = soup.find_all("div", {"class": "field field-name-field-deaths field-type-number-integer field-label-above"})

    states = []
    for val in STATES:
        states.append(str(val.getText()).split(":")[1].lstrip().replace("Telengana", "Telangana").replace('Andaman And Nicobar', 'Andaman and Nicobar Islands').replace('Andaman and Nicobar','Andaman and Nicobar Islands'))

    confirmed = []
    for val in CONFIRMED:
        confirmed.append(str(val.getText()).split(":")[1].lstrip())

    cured = []
    for val in CURED_DISCHARGED:
        cured.append(str(val.getText()).split(":")[1].lstrip())

    death = []
    for val in DEATH:
        death.append(str(val.getText()).split(":")[1].lstrip())

    states_data = pd.DataFrame(list(zip(states, confirmed, cured, death)))

    final_df_col = [
        'Date', 'State/UTCode', 'District', 'tested_last_updated_district', 'tested_source_district',
        'notesForDistrict', 'cumulativeConfirmedNumberForDistrict', 'cumulativeDeceasedNumberForDistrict',
        'cumulativeRecoveredNumberForDistrict', 'cumulativeTestedNumberForDistrict',
        'cumulativeVaccinatedNumberForDistrict', 'last_updated', 'tested_last_updated_state', 'tested_source_state',
        'notesForState', 'cumulativeConfirmedNumberForState', 'cumulativeDeceasedNumberForState',
        'cumulativeRecoveredNumberForState', 'cumulativeTestedNumberForState', 'cumulativeVaccinatedNumberForState'
    ]

    states_data = states_data.rename(
        columns={0: "District", 1: "cumulativeConfirmedNumberForDistrict", 2: "cumulativeRecoveredNumberForDistrict",
                 3: "cumulativeDeceasedNumberForDistrict"})

    states_data["Date"] = date#str(datetime.datetime.now().date())

    states_data["State/UTCode"] = state

    states_data['cumulativeConfirmedNumberForState'] = int(
        sum_soup.findAll("div", {"class": "t_case"})[0].findAll("span", {"class": "icount"})[0].getText().replace(
            ",", ""))

    states_data['cumulativeDeceasedNumberForState'] = int(
        sum_soup.findAll("div", {"class": "death_case"})[0].findAll("span", {"class": "icount"})[0].getText().replace(
            ",", ""))

    states_data['cumulativeRecoveredNumberForState'] = int(
        sum_soup.findAll("div", {"class": "discharge"})[0].findAll("span", {"class": "icount"})[0].getText().replace(
            ",", ""))
    
    
    states_data["last_updated"] = str(datetime.datetime.now())

    states_data['cumulativeTestedNumberForState'] = int(
        sum_soup.findAll("div", {"class": "testing_result"})[0].findAll("strong")[0].getText().replace(",", ""))
    
    
    states_data['tested_last_updated_state'] = sum_soup.findAll(
        "div", {"class": "test_title"})[0].getText().strip().split("up to ")[-1]
    
    states_data['deltaTestedForState'] = int(
        sum_soup.findAll("div", {"class": "testing_sample"})[0].findAll("strong")[0].getText().replace(",", ""))
    
    for vcount in sum_soup.findAll("div", {"class": "total-vcount"}):
        if 'yday' not in vcount.attrs['class']:
            states_data['cumulativeVaccinatedNumberForState'] = int(vcount.findAll("strong")[0].getText().replace(",", ""))

    # states_data = states_data.reindex(columns=final_df_col)
    return states_data

In [79]:
today = (datetime.datetime.now() - timedelta.Timedelta(days=0)).date()
pDate = (datetime.datetime.now() - timedelta.Timedelta(days=1)).date()
cowinDate = (datetime.datetime.now() - timedelta.Timedelta(days=2)).date()
print(str(today))
print(str(pDate))

TT_df = india("TT",str(today))

# ASHA .. For Running previous dates
# date_time_str = '2021/11/06'
# date_time_obj = datetime.datetime.strptime(date_time_str, '%Y/%m/%d')
# print(str(date_time_obj.date()))
# today = (date_time_obj - timedelta.Timedelta(days=0)).date()
# pDate = (date_time_obj - timedelta.Timedelta(days=1)).date()
# print(str(today))
# print(str(pDate))
# TT_df = india("TT",str(today))
# ASHA .. For Running previous dates



2022-01-16
2022-01-15


In [80]:
if not os.path.isdir(os.path.join("..","RAWCSV",str(pDate),"myGov")):
    os.mkdir(os.path.join("..","RAWCSV",str(pDate),"myGov"))
    print("Created")

Created


In [81]:
#mygov of prevDate
from ExtractStateMyGov import ExtractStateMyGov
source = pd.read_csv(r"../sources.csv")
for idx in source.index:
    if source["StateCode"][idx] != "TT":
        print(source["StateCode"][idx])
        if source["myGov"][idx] == "yes":
            temp_df = ExtractStateMyGov(source["StateCode"][idx],str(today))
        else:
            temp_df = ExtractStateMyGov(source["StateCode"][idx],str(today), no_source = True)
        temp_df["Date"] = pDate
        temp_df.to_csv(os.path.join("..","RAWCSV",str(pDate),"myGov",source["StateCode"][idx]+"_raw.csv"))

AP
AR
AS


/home/swiadmin/Incovid19/incovid19/src/ExtractStateMyGov.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df["cumulativeConfirmedNumberForState"] = state_df['cumulativeConfirmedNumberForDistrict'][state_df['District'] == 'Unknown'] = int(ind["cumulativeConfirmedNumberForDistrict"][ind["District"].str.contains(state_name)].values[0])
/home/swiadmin/Incovid19/incovid19/src/ExtractStateMyGov.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df["cumulativeDeceasedNumberForState"] = state_df['cumulativeDeceasedNumberForDistrict'][state_df['District'] == 'Unknown'] = int(ind["cumulativeDeceasedNumberForDis

BR
CT
GA
GJ
HR
HP
JH
KA
KL
MP
MH
MN
ML
MZ
NL


/home/swiadmin/Incovid19/incovid19/src/ExtractStateMyGov.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][df['District'] == district] = None


OR
PB
RJ
SK
TN
TG
TR
UP
UT
WB
AN
CH
DN
DL
LA
LD
JK
PY


In [82]:
# TT_df = pd.read_csv("../RAWCSV/2021-10-31/TT_raw.csv")

In [83]:
TT_df = TT_df.dropna(1)

In [84]:
import urllib.request, json 
with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(pDate)) as url:
    data = json.loads(url.read().decode())

In [85]:
TT_df["cumulativeVaccinated1NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_1"]
TT_df["cumulativeVaccinated2NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_2"]
TT_df["cumulativeVaccinatedNumberForState"] = data["topBlock"]["vaccination"]["total_doses"]

In [86]:
print(TT_df["cumulativeVaccinated1NumberForState"][0])
print(TT_df["cumulativeVaccinated2NumberForState"][0])
print(TT_df["cumulativeVaccinatedNumberForState"][0])

909005449
654485860
1567676736


In [87]:
states = []
vaccinated = []
vaccinated1 = []
vaccinated2 = []
for sta in data['getBeneficiariesGroupBy']:
    if (sta['title'].lower() == "dadra and nagar haveli") or (sta['title'].lower() == "daman and diu"):
        states.append('Dadra and Nagar Haveli and Daman and Diu')
    else:
        states.append(sta['title'])
    vaccinated.append(sta['total'])
    vaccinated1.append(sta['partial_vaccinated'])
    vaccinated2.append(sta['totally_vaccinated'])

In [88]:
VAC_df = pd.DataFrame(list(zip(states,vaccinated,vaccinated1,vaccinated2))).groupby(0, as_index=False).sum()

In [89]:
VAC_df = VAC_df.rename(columns={0:"District",1:"cumulativeVaccinatedNumberForDistrict",2:"cumulativeVaccinated1NumberForDistrict",3:"cumulativeVaccinated2NumberForDistrict",})

In [90]:
TT_df = TT_df.merge(VAC_df,how="left",on="District")

In [91]:
TT_df["tested_source_state"] = "https://www.icmr.gov.in/"

In [92]:
# TT_df.to_csv("TT_test.csv")

In [93]:
TT_df["tested_last_updated_state"] = str(datetime.datetime.strptime(TT_df["tested_last_updated_state"][0] , '%b %d, %Y'))

# Generate TT_final

In [94]:
# cs = pd.read_csv("case_time_series.csv")

In [95]:
# cs.columns

In [96]:
# cs.tail()

In [97]:
# import timedelta
# delta_date = str((datetime.datetime.now() - timedelta.Timedelta(days=1)).date())

In [98]:
delta_date = str(pDate)

In [99]:
delta_date

'2022-01-15'

In [100]:
# ind = cs[cs['Date_YMD']==delta_date].index[0]

# Integrate is CS only

In [101]:
# states_df = pd.read_csv("states.csv")

In [102]:
# cs = states_df

In [103]:
# ind = cs[(cs['Date']==delta_date) & (cs["State"] == "India")].index[0]

In [104]:
# ind

# Reading from Raw

In [105]:
cs = pd.read_csv("../RAWCSV/"+delta_date+"/TT_final.csv")

In [106]:
# cs.columns

In [107]:
# TT_df["deltaConfirmedForState"] = TT_df["cumulativeConfirmedNumberForState"] - cs["Confirmed"][ind]
# TT_df["deltaDeceasedForState"] = TT_df["cumulativeDeceasedNumberForState"] - cs["Deceased"][ind]
# TT_df["deltaRecoveredForState"] = TT_df["cumulativeRecoveredNumberForState"] - cs["Recovered"][ind]
TT_df["deltaConfirmedForState"] = TT_df["cumulativeConfirmedNumberForState"] - cs["cumulativeConfirmedNumberForState"][0]
TT_df["deltaDeceasedForState"] = TT_df["cumulativeDeceasedNumberForState"] - cs["cumulativeDeceasedNumberForState"][0]
TT_df["deltaRecoveredForState"] = TT_df["cumulativeRecoveredNumberForState"] - cs["cumulativeRecoveredNumberForState"][0]
TT_df["deltaTestedForState"] = TT_df["cumulativeTestedNumberForState"] - cs["cumulativeTestedNumberForState"][0]

In [108]:
delta_date

'2022-01-15'

In [109]:
import urllib.request, json 
with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(cowinDate)) as url:
    data = json.loads(url.read().decode())

In [110]:
# TT_df["cumulativeVaccinated1NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_1"]
# TT_df["cumulativeVaccinated2NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_2"]
# TT_df["cumulativeVaccinatedNumberForState"] = data["topBlock"]["vaccination"]["total_doses"]

In [111]:
print(data["topBlock"]["vaccination"]["tot_dose_2"])
print(data["topBlock"]["vaccination"]["tot_dose_1"])
print(data["topBlock"]["vaccination"]["total_doses"])

650915816
905748499
1560367967


In [112]:
TT_df["deltaVaccinated2ForState"] = TT_df["cumulativeVaccinated2NumberForState"] - data["topBlock"]["vaccination"]["tot_dose_2"]
TT_df["deltaVaccinated1ForState"] = TT_df["cumulativeVaccinated1NumberForState"] - data["topBlock"]["vaccination"]["tot_dose_1"]
TT_df["deltaVaccinatedForState"] = TT_df["cumulativeVaccinatedNumberForState"] - data["topBlock"]["vaccination"]["total_doses"]

In [113]:
print(TT_df["deltaVaccinated1ForState"][0])
print(TT_df["deltaVaccinated2ForState"][0])
print(TT_df["deltaVaccinatedForState"][0])

3256950
3570044
7308769


In [114]:
# TT_df["deltaVaccinatedForState"]

In [115]:
states = []
vaccinated = []
vaccinated1 = []
vaccinated2 = []
for sta in data['getBeneficiariesGroupBy']:
    if (sta['title'].lower() == "dadra and nagar haveli") or (sta['title'].lower() == "daman and diu"):
        states.append('Dadra and Nagar Haveli and Daman and Diu')
    else:
        states.append(sta['title'])
    vaccinated.append(sta['total'])
    vaccinated1.append(sta['partial_vaccinated'])
    vaccinated2.append(sta['totally_vaccinated'])

In [116]:
prevdayVACC_df = pd.DataFrame(list(zip(states,vaccinated,vaccinated1,vaccinated2))).groupby(0, as_index=False).sum()

In [117]:
prevdayVACC_df = prevdayVACC_df.rename(columns={0:"District",1:"cumulativeVaccinatedNumberForDistrict",2:"cumulativeVaccinated1NumberForDistrict",3:"cumulativeVaccinated2NumberForDistrict",})

In [118]:
# TT_df.to_csv("in_test.csv")

In [119]:
lst = ["Vaccinated1","Vaccinated2","Vaccinated"]
for val in lst:
    TT_df["delta{}ForDistrict".format(val)] = None
    for idx in TT_df.index:
        TT_df["delta{}ForDistrict".format(val)][idx] = TT_df["cumulative{}NumberForDistrict".format(val)][idx] - prevdayVACC_df[prevdayVACC_df["District"] == TT_df["District"][idx]]["cumulative{}NumberForDistrict".format(val)].item()

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [120]:
# states_df = pd.read_csv("states.csv")

# No reading states

In [121]:
states_df = cs[["Date","District","cumulativeConfirmedNumberForDistrict","cumulativeRecoveredNumberForDistrict","cumulativeDeceasedNumberForDistrict"]]

In [122]:
states_df

,Date,District,cumulativeConfirmedNumberForDistrict,cumulativeRecoveredNumberForDistrict,cumulativeDeceasedNumberForDistrict
0,2022-01-15,Andaman and Nicobar Islands,8538,7938,129
1,2022-01-15,Andhra Pradesh,2096755,2063934,14508
2,2022-01-15,Arunachal Pradesh,56227,55108,282
3,2022-01-15,Assam,640636,617825,6197
4,2022-01-15,Bihar,775392,729184,12123
5,2022-01-15,Chandigarh,74475,66132,1086
6,2022-01-15,Chhattisgarh,1050228,1005727,13639
7,2022-01-15,Dadra and Nagar Haveli and Daman and Diu,10911,10731,4
8,2022-01-15,Delhi,1670966,1553388,25305
9,2022-01-15,Goa,203969,181826,3546


In [123]:
# deltaConfirmedForDistrict",
#                  "deltaDeceasedForDistrict",
#                  "deltaRecoveredForDistrict",
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    TT_df["delta{}ForDistrict".format(val)] = None
    for idx in TT_df.index:
        TT_df["delta{}ForDistrict".format(val)][idx] = int(TT_df["cumulative{}NumberForDistrict".format(val)][idx]) - int(states_df[(states_df["District"] == TT_df["District"][idx]) & (states_df["Date"] == delta_date)]["cumulative{}NumberForDistrict".format(val)].item())

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [124]:
delta_date

'2022-01-15'

In [125]:
lst = ["Vaccinated1","Vaccinated2","Vaccinated"]
for val in lst:
    TT_df["delta{}ForDistrict".format(val)] = None
    for idx in TT_df.index:
        TT_df["delta{}ForDistrict".format(val)][idx] = TT_df["cumulative{}NumberForDistrict".format(val)][idx] - prevdayVACC_df[prevdayVACC_df["District"] == TT_df["District"][idx]]["cumulative{}NumberForDistrict".format(val)].item()

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [126]:
# TT_df 7dma Cummulative

In [127]:
cs.tail()

,Unnamed: 0,District,cumulativeConfirmedNumberForDistrict,cumulativeRecoveredNumberForDistrict,cumulativeDeceasedNumberForDistrict,Date,State/UTCode,cumulativeConfirmedNumberForState,cumulativeDeceasedNumberForState,cumulativeRecoveredNumberForState,last_updated,cumulativeTestedNumberForState,tested_last_updated_state,deltaTestedForState,cumulativeVaccinatedNumberForState,cumulativeVaccinated1NumberForState,cumulativeVaccinated2NumberForState,cumulativeVaccinatedNumberForDistrict,cumulativeVaccinated1NumberForDistrict,cumulativeVaccinated2NumberForDistrict,tested_source_state,deltaConfirmedForState,deltaDeceasedForState,deltaRecoveredForState,deltaVaccinated2ForState,deltaVaccinated1ForState,deltaVaccinatedForState,deltaVaccinated1ForDistrict,deltaVaccinated2ForDistrict,deltaVaccinatedForDistrict,deltaConfirmedForDistrict,deltaRecoveredForDistrict,deltaDeceasedForDistrict,7DmaConfirmedForState,7DmaRecoveredForState,7DmaDeceasedForState,7DmaVaccinated1ForState,7DmaVaccinated2ForState,7DmaVaccinatedForState,7DmaConfirmedForDistrict,7DmaRecoveredForDistrict,7DmaDeceasedForDistrict,7DmaVaccinatedForDistrict,7DmaVaccinated1ForDistrict,7DmaVaccinated2ForDistrict,7DmaTestedForState,delta21_14confirmedForState,statePopulation,StateName,districtPopulation
31,31,Telangana,705199,679471,4052,2022-01-15,TT,36850962,485752,34947390,2022-01-15 12:25:01.706693,700712824,2022-01-14 00:00:00,1613740,1560367967,905748499,650915816,50250970,29407672,20723406,https://www.icmr.gov.in/,268833,402,122684,3234099,2626525,6267056,17411,43798,66241,2398,1181,3,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363567e+06,1.340221e+06,16386.142857,0,0,0,NaN,NaN,1332900000,Telangana,37220000
32,32,Tripura,89767,84800,835,2022-01-15,TT,36850962,485752,34947390,2022-01-15 12:25:01.706693,700712824,2022-01-14 00:00:00,1613740,1560367967,905748499,650915816,4858072,2697114,2149529,https://www.icmr.gov.in/,268833,402,122684,3234099,2626525,6267056,274,194,1177,546,170,3,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363567e+06,1.340221e+06,16386.142857,0,0,0,NaN,NaN,1332900000,Tripura,3992000
33,33,Uttar Pradesh,1801231,1693842,22949,2022-01-15,TT,36850962,485752,34947390,2022-01-15 12:25:01.706693,700712824,2022-01-14 00:00:00,1613740,1560367967,905748499,650915816,225854399,141055562,84468799,https://www.icmr.gov.in/,268833,402,122684,3234099,2626525,6267056,1274763,1328503,2664182,15975,2554,3,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363567e+06,1.340221e+06,16386.142857,0,0,0,NaN,NaN,1332900000,Uttar Pradesh,224979000
34,34,Uttarakhand,363424,343637,7438,2022-01-15,TT,36850962,485752,34947390,2022-01-15 12:25:01.706693,700712824,2022-01-14 00:00:00,1613740,1560367967,905748499,650915816,15082578,8315283,6696288,https://www.icmr.gov.in/,268833,402,122684,3234099,2626525,6267056,9349,14725,32728,3200,784,3,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363567e+06,1.340221e+06,16386.142857,0,0,0,NaN,NaN,1332900000,Uttarakhand,11141000
35,35,West Bengal,1863697,1698201,20013,2022-01-15,TT,36850962,485752,34947390,2022-01-15 12:25:01.706693,700712824,2022-01-14 00:00:00,1613740,1560367967,905748499,650915816,113223969,67909159,45107196,https://www.icmr.gov.in/,268833,402,122684,3234099,2626525,6267056,191060,445824,680947,22645,8687,28,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363567e+06,1.340221e+06,16386.142857,80598793,56330081,24239052,NaN,NaN,1332900000,West Bengal,96906000


In [128]:
cs = pd.read_csv("case_time_series.csv")

In [129]:
states_df = pd.read_csv("states.csv")

# Remenber to read CS and states

In [130]:
from datetime import datetime, timedelta
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    value = 0
    for i in range(7):
        if i == 0:
            value = TT_df["cumulative{}NumberForState".format(val)]
        else:
            prev_date=datetime.strptime("2021-10-31","%Y-%m-%d")+timedelta(days=(-1*i))
            ind = cs[cs['Date_YMD']==str(prev_date.date())].index[0]
            value =+ cs["Total {}".format(val)][ind]
    TT_df["7Dma{}ForState".format(val)] = value/7

In [131]:
import urllib.request, json 
va1 = 0
va2 = 0
va = 0
for i in range(7):
    prev_date=datetime.strptime("2021-10-31","%Y-%m-%d")+timedelta(days=(-1*i))
    with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(prev_date.date())) as url:
        data = json.loads(url.read().decode())
        va1 += data["topBlock"]["vaccination"]["tot_dose_1"]
        va2 += data["topBlock"]["vaccination"]["tot_dose_2"]
        va += data["topBlock"]["vaccination"]["total_doses"]


In [132]:
TT_df["7DmaVaccinated1ForState"] = va1
TT_df["7DmaVaccinated2ForState"] = va2
TT_df["7DmaVaccinatedForState"] = va

In [133]:
# 7dma District

In [134]:
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    value = 0
    for idx in TT_df.index:
        for i in range(7):
            if i == 0:
                value = int(TT_df["cumulative{}NumberForDistrict".format(val)][0])
            else:
                prev_date=datetime.strptime("2021-10-31","%Y-%m-%d")+timedelta(days=(-1*i))
                value += int(states_df[(states_df["State"] == TT_df["District"][idx]) & (states_df["Date"] == str(prev_date.date()))]['{}'.format(val)].item())
    TT_df["7Dma{}ForDistrict".format(val)] = value/7          

In [135]:
cowin = pd.read_csv("cowin_vaccine_data_statewise.csv")

In [136]:
va1 = 0
va2 = 0
va = 0
TT_df["7DmaVaccinatedForDistrict"] = 0
TT_df["7DmaVaccinated1ForDistrict"] = 0
TT_df["7DmaVaccinated2ForDistrict"] = 0
for idx in TT_df.index:
    for i in range(7):
        if i == 0:
            va = int(TT_df["cumulativeVaccinatedNumberForDistrict"][idx])
            va1 = int(TT_df["cumulativeVaccinated1NumberForDistrict"][idx])
            va2 = int(TT_df["cumulativeVaccinated2NumberForDistrict"][idx])
        else:
            prev_date=datetime.strptime("2021/10/31","%Y/%m/%d")+timedelta(days=(-1*i))
            va += int(cowin[(cowin["Updated On"] ==  prev_date.strftime('%d/%m/%Y')) & (cowin["State"] == TT_df["District"][idx])]["Total Doses Administered"].item())
            va1 += int(cowin[(cowin["Updated On"] ==  prev_date.strftime('%d/%m/%Y')) & (cowin["State"] == TT_df["District"][idx])]["First Dose Administered"].item())
            va2 += int(cowin[(cowin["Updated On"] ==  prev_date.strftime('%d/%m/%Y')) & (cowin["State"] == TT_df["District"][idx])]["Second Dose Administered"].item())
TT_df["7DmaVaccinatedForDistrict"][idx] = va/7
TT_df["7DmaVaccinated1ForDistrict"][idx] = va1/7
TT_df["7DmaVaccinated2ForDistrict"][idx] = va2/7

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [137]:
TT_df["7DmaTestedForState"] = None
TT_df["delta21_14confirmedForState"] = None

In [138]:
population = pd.read_csv("../CSV/StatePopulation.csv")

In [139]:
sources = pd.read_csv("../sources.csv")

In [140]:
population = population.merge(sources,how="left",left_on = "State", right_on = "StateCode")

In [141]:
population

,State,StatePop,StateName,StateCode,StateDataSourceType,myGov,StateDataURL,Twitter Handle,Twitter Search Term
0,AN,397000,Andaman and Nicobar Islands,AN,html,yes,https://www.mygov.in/covid-19,NaN,NaN
1,AP,52221000,Andhra Pradesh,AP,html,NaN,https://hmfw.ap.gov.in/covid_dashboard.aspx,NaN,NaN
2,AR,1504000,Arunachal Pradesh,AR,Image(Twitter),NaN,https://twitter.com/DirHealth_ArPr,DirHealth_ArPr,#ArunachalCoronaUpdate
3,AS,34293000,Assam,AS,html,yes,https://www.mygov.in/covid-19,NaN,NaN
4,BR,119520000,Bihar,BR,Image(Twitter),NaN,https://twitter.com/BiharHealthDept,BiharHealthDept,#COVIDー19 Updates Bihar
5,CH,1179000,Chandigarh,CH,html,yes,https://www.mygov.in/covid-19,NaN,NaN
6,CT,28724000,Chhattisgarh,CT,Image(Twitter),NaN,https://twitter.com/HealthCgGov,HealthCgGov,#ChhattisgarhFightsCorona
7,DL,19814000,Delhi,DL,html,yes,https://www.mygov.in/covid-19,NaN,NaN
8,DN,959000,Dadra and Nagar Haveli and Daman and Diu,DN,html,yes,https://www.mygov.in/covid-19,NaN,NaN
9,GA,1540000,Goa,GA,html,yes,https://www.mygov.in/covid-19,NaN,NaN


In [142]:
population[["StateName","StatePop"]]

,StateName,StatePop
0,Andaman and Nicobar Islands,397000
1,Andhra Pradesh,52221000
2,Arunachal Pradesh,1504000
3,Assam,34293000
4,Bihar,119520000
5,Chandigarh,1179000
6,Chhattisgarh,28724000
7,Delhi,19814000
8,Dadra and Nagar Haveli and Daman and Diu,959000
9,Goa,1540000


In [143]:
TT_df["statePopulation"] = population[population["StateName"] == "India"]["StatePop"].item()

In [144]:
TT_df

,District,cumulativeConfirmedNumberForDistrict,cumulativeRecoveredNumberForDistrict,cumulativeDeceasedNumberForDistrict,Date,State/UTCode,cumulativeConfirmedNumberForState,cumulativeDeceasedNumberForState,cumulativeRecoveredNumberForState,last_updated,cumulativeTestedNumberForState,tested_last_updated_state,deltaTestedForState,cumulativeVaccinatedNumberForState,cumulativeVaccinated1NumberForState,cumulativeVaccinated2NumberForState,cumulativeVaccinatedNumberForDistrict,cumulativeVaccinated1NumberForDistrict,cumulativeVaccinated2NumberForDistrict,tested_source_state,deltaConfirmedForState,deltaDeceasedForState,deltaRecoveredForState,deltaVaccinated2ForState,deltaVaccinated1ForState,deltaVaccinatedForState,deltaVaccinated1ForDistrict,deltaVaccinated2ForDistrict,deltaVaccinatedForDistrict,deltaConfirmedForDistrict,deltaRecoveredForDistrict,deltaDeceasedForDistrict,7DmaConfirmedForState,7DmaRecoveredForState,7DmaDeceasedForState,7DmaVaccinated1ForState,7DmaVaccinated2ForState,7DmaVaccinatedForState,7DmaConfirmedForDistrict,7DmaRecoveredForDistrict,7DmaDeceasedForDistrict,7DmaVaccinatedForDistrict,7DmaVaccinated1ForDistrict,7DmaVaccinated2ForDistrict,7DmaTestedForState,delta21_14confirmedForState,statePopulation
0,Andaman and Nicobar Islands,8581,7989,129,2022-01-16,TT,37122164,486066,35085721,2022-01-16 12:01:14.771605,702448838,2022-01-15 00:00:00,1736014,1567676736,909005449,654485860,621942,321766,297918,https://www.icmr.gov.in/,271202,314,138331,3570044,3256950,7308769,744,356,1643,43,51,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363573e+06,1.340228e+06,16386.142857,0,0,0,None,None,1332900000
1,Andhra Pradesh,2101710,2064331,14509,2022-01-16,TT,37122164,486066,35085721,2022-01-16 12:01:14.771605,702448838,2022-01-15 00:00:00,1736014,1567676736,909005449,654485860,75218714,42327680,32630324,https://www.icmr.gov.in/,271202,314,138331,3570044,3256950,7308769,2190,28881,32610,4955,397,1,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363573e+06,1.340228e+06,16386.142857,0,0,0,None,None,1332900000
2,Arunachal Pradesh,56493,55141,282,2022-01-16,TT,37122164,486066,35085721,2022-01-16 12:01:14.771605,702448838,2022-01-15 00:00:00,1736014,1567676736,909005449,654485860,1524869,853909,664913,https://www.icmr.gov.in/,271202,314,138331,3570044,3256950,7308769,3029,1650,5846,266,33,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363573e+06,1.340228e+06,16386.142857,0,0,0,None,None,1332900000
3,Assam,644026,618701,6201,2022-01-16,TT,37122164,486066,35085721,2022-01-16 12:01:14.771605,702448838,2022-01-15 00:00:00,1736014,1567676736,909005449,654485860,39783380,22857775,16884923,https://www.icmr.gov.in/,271202,314,138331,3570044,3256950,7308769,10725,30681,44253,3390,876,4,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363573e+06,1.340228e+06,16386.142857,0,0,0,None,None,1332900000
4,Bihar,781717,733673,12127,2022-01-16,TT,37122164,486066,35085721,2022-01-16 12:01:14.771605,702448838,2022-01-15 00:00:00,1736014,1567676736,909005449,654485860,106809961,62028149,44570466,https://www.icmr.gov.in/,271202,314,138331,3570044,3256950,7308769,187095,199939,407261,6325,4489,4,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363573e+06,1.340228e+06,16386.142857,0,0,0,None,None,1332900000
5,Chandigarh,76270,66673,1086,2022-01-16,TT,37122164,486066,35085721,2022-01-16 12:01:14.771605,702448838,2022-01-15 00:00:00,1736014,1567676736,909005449,654485860,1923376,1089217,828264,https://www.icmr.gov.in/,271202,314,138331,3570044,3256950,7308769,4115,2763,7800,1795,541,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363573e+06,1.340228e+06,16386.142857,0,0,0,None,None,1332900000
6,Chhattisgarh,1055753,1009967,13647,2022-01-16,TT,37122164,486066,35085721,2022-01-16 12:01:14.771605,702448838,2022-01-15 00:00:00,1736014,1567676736,909005449,654485860,32236968,19031349,13088860,https://www.icmr.gov.in/,271202,314,138331,3570044,

In [145]:
TT_df = TT_df.merge(population[["StateName","StatePop"]],how="left",left_on = "District", right_on = "StateName")

In [146]:
TT_df = TT_df.rename(columns={"StatePop":"districtPopulation"})

In [147]:
today

datetime.date(2022, 1, 16)

In [148]:
TT_df["Date"] = str(today)

In [149]:
TT_df.to_csv('../RAWCSV/'+str(today)+'/TT_final.csv')

In [150]:
TT_df.to_csv('../RAWCSV/'+str(today)+'/TT_final-'+str(today)+'.csv')

In [151]:
TT_df["Date"] = str(pDate)

In [152]:
TT_df.to_csv('../RAWCSV/'+str(pDate)+'/TT_final.csv')

# Stop Here

In [151]:
# 7dma delta

In [152]:
states_df[states_df["State"] == "India"]

,Date,State,Confirmed,Recovered,Deceased,Other,Tested
1,2020-01-30,India,1,0,0,0,NaN
3,2020-02-02,India,2,0,0,0,NaN
5,2020-02-03,India,3,0,0,0,NaN
7,2020-02-14,India,3,3,0,0,NaN
11,2020-03-02,India,5,3,0,0,NaN
16,2020-03-03,India,6,3,0,0,NaN
22,2020-03-04,India,28,3,0,0,NaN
29,2020-03-05,India,30,3,0,0,NaN
36,2020-03-06,India,31,3,0,0,NaN
45,2020-03-07,India,34,3,0,0,NaN


In [298]:
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    value = []
    for i in range(7):
        if i == 0:
            value.append(TT_df["cumulative{}NumberForState".format(val)][0])
        else:
            prev_date=datetime.strptime("2021-11-01","%Y-%m-%d")+timedelta(days=(-1*i))
            value.append(states_df[(states_df["State"] == "India") & (states_df["Date"] == str(prev_date.date()))]['{}'.format(val)].item())

In [299]:
value

[458437, 458470, 458219, 457774, 457223, 456418, 455684]